# Initialisation: Revision Record

In [1]:
import sys
from pathlib import Path

# Find the nearest directory that has a "src" folder (works no matter where you run the notebook)
here = Path.cwd()
src_dir = next(p for p in [here, *here.parents] if (p / "src").exists()) / "src"
sys.path.insert(0, str(src_dir))

In [2]:
# a block primarily for handling `.idf` files from `Rhino + Grasshopper` via `Ladybug Tools`
from ReIDF import relocate_IDF, md_ghIDF, md_IDFsch2series, md_IDFsch4gh, md_IDFdt, md_IDFop_ctr, ReIDF1

source_directory_1 = R"C:\Users\s2589602\0.Recents\1.Rhino\GH-outputs\HVACDetailed-Oct-13\openstudio\run"
dest_folder_name='idf-Oct'

paths = [] # KEY-1

for source_directory in [source_directory_1]:
    path=relocate_IDF(source_directory, dest_folder_name)
    md_ghIDF(path, year=2023)
    
    # Universal functions
    md_IDFsch2series(path)
    md_IDFsch4gh(path)
    md_IDFdt(path, '2023-01-01', '2023-12-31')
    md_IDFop_ctr(path, ['CSV', 'MTR', 'Tabular'])

    paths.append(path) # KEY-2

File copied and renamed to:
c:\Users\s2589602\Downloads\2025\src\eplus_heat_balance\idf-Oct\HVACDetailed-Oct-13.idf
idf-Oct/HVACDetailed-Oct-13.idf
Modified idf-Oct/HVACDetailed-Oct-13.idf


# 1. Key Revisions

In [3]:
from ReIDF import ReIDF1, md_IDFop_ctr

# clg_vrf_updated_='23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23;'
# clg_office_updated_='35, 35, 35, 35, 35, 35, 35, 35, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 35, 35, 35, 35, 35;'
# clg_con_office_updated_='35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35;'

ReClass_HVAC = [
    # ('ZoneVentilation:WindandStackOpenArea', 3, 2),
    # ('ZoneVentilation:WindandStackOpenArea', 7, 2),
    ('Fan:OnOff', 5, 0.33),
    # ('Fan:ConstantVolume', 5, 0.2163)
]
#0.4153
ReUnit_HVAC = [
    # ('inftr_atrium_br_',7, 0.00125),
    ({'Coil:Heating:Water': '26 Zone DOAS Htg Coil'}, {3: '', 4: '', 9: 'NominalCapacity', 10: 150000}),
    ({'Coil:Heating:Water': '35 Zone DOAS Htg Coil'}, {3: '', 4: '', 9: 'NominalCapacity', 10: 155000}),
    ({'Coil:Heating:Water': '1 Zone DOAS Htg Coil'}, {3: '', 4: '', 9: 'NominalCapacity', 10: 49000}),
    ({'Coil:Heating:Water': '63 Zone DOAS Htg Coil'}, {3: '', 4: '', 9: 'NominalCapacity', 10: 155000}),
    # ({'Fan:ConstantVolume':'DOAS Exhaust Fan'}, 5, 0.4153), 
    # ({'Fan:ConstantVolume':'DOAS Exhaust Fan 1'}, 5, 0.4153), 
    # ({'Fan:ConstantVolume':'DOAS Exhaust Fan 2'}, 5, 0.4153),    
    # ('clg_sch_office_br__Hol 1',3, f'{clg_con_office_updated_}'),         
]

idf_variants = ['HVACDetailed', 'HVACLumped', 'HVACIdeal']
updatedIDFs = []

for variant in idf_variants:
    ori_path = f'idf-Oct/{variant}-Oct-13.idf'
    cus_path = f'idf-Oct/{variant}-Oct-13_.idf'
    updated_idf = ReIDF1(
        oriPath=ori_path,
        ReTemp=None,
        ReClass=ReClass_HVAC,
        ReUnit=ReUnit_HVAC,
        cusPath=cus_path
    )
    updatedIDFs.append(updated_idf)

ReIDF: → C:\Users\s2589602\Downloads\2025\src\eplus_heat_balance\idf-Oct\HVACDetailed-Oct-13_.idf
No modifications made for unit '63 Zone DOAS Htg Coil'
ReUnit: [{"{'Coil:Heating:Water': '26 Zone DOAS Htg Coil'}": 4}, {"{'Coil:Heating:Water': '35 Zone DOAS Htg Coil'}": 4}, {"{'Coil:Heating:Water': '1 Zone DOAS Htg Coil'}": 4}, {"{'Coil:Heating:Water': '63 Zone DOAS Htg Coil'}": 0}]
ReClass: [{'Fan:OnOff': 67}]
Copy error: [Errno 2] No such file or directory: 'C:\\Users\\s2589602\\Downloads\\2025\\src\\eplus_heat_balance\\idf-Oct\\HVACLumped-Oct-13.idf'
Copy error: [Errno 2] No such file or directory: 'C:\\Users\\s2589602\\Downloads\\2025\\src\\eplus_heat_balance\\idf-Oct\\HVACIdeal-Oct-13.idf'


In [4]:
idf_file = updatedIDFs[0]
md_IDFop_ctr(idf_file, ['CSV', 'MTR', 'MDD', 'RDD'])

# 3. Executing EnergyPlus
* For `mac` operations, using files:

    weather_file = '/Users/rui.bo/Downloads/EnergyPlus/GBR_SCT_Edinburgh.AP.031600_TMYx.2023.epw'

    energyplus_exe = '/Applications/EnergyPlus-23-2-0/EnergyPlus'

* For `windows` operations, using files:

    energyplus_exe = 'C:/EnergyPlusV23-2-0/EnergyPlus'

    weather_file = 'C:/Users/s2589602/Downloads/a2-May/EPlus/EPlus/epw/GBR_SCT_Edinburgh.AP.031600_TMYx.2023.epw'

(1) Single File

In [5]:
from ReIDF import run_IDF
weather_file = '../_base/epw/ED-TMYx.2023.epw'
energyplus_exe = 'C:/EnergyPlusV23-2-0/EnergyPlus'

idf_file=updatedIDFs[0]

output_folder = run_IDF(
    idf_file=idf_file,
    weather_file=weather_file,
    energyplus_exe=energyplus_exe
)

Simulation completed successfully.
Output Folder: C:\Users\s2589602\Downloads\2025\src\eplus_heat_balance\idf-Oct\HVACDetailed-Oct-13_


(2) Group File

In [6]:
import pandas as pd
from pathlib import Path

def relabel_energyplus(
    csv_in,
    csv_out=None,
    mode="calendar",          # "calendar" or "index"
    start="2023-01-01",       # used in calendar mode
    freq="H",                 # e.g. "H" (hourly), "10min", "30min"
    expected_periods=None,    # e.g. 8760, 8784, or 365*24
    keep_original_datetime=False  # set True to keep original "Date/Time"
):
    """
    Read an EnergyPlus CSV, drop the text 'Date/Time' column, and add a user-defined timestamp.
    If expected_periods is given and the file has extra rows (e.g., design/warmup days),
    the function keeps only the last `expected_periods` rows.
    """
    csv_in = Path(csv_in)

    # --- Find the header row that contains 'Date/Time' ---
    header_row = None
    with csv_in.open("r", encoding="utf-8", errors="ignore") as f:
        for i, line in enumerate(f):
            # normalize commas vs values; we only need to detect the header line
            if "Date/Time" in line.split(","):
                header_row = i
                break
    if header_row is None:
        raise ValueError("Could not find a header row containing 'Date/Time'.")

    # --- Read using the detected header row ---
    df = pd.read_csv(csv_in, header=header_row)

    # --- Optionally trim to expected periods (drop warmup/design days) ---
    if expected_periods is not None:
        if len(df) >= expected_periods:
            df = df.tail(expected_periods).copy()
        else:
            raise ValueError(
                f"File has only {len(df)} rows, fewer than expected_periods={expected_periods}."
            )

    # --- Drop the original Date/Time text column (optional keep) ---
    if "Date/Time" in df.columns and not keep_original_datetime:
        df = df.drop(columns=["Date/Time"])

    # --- Build the new timestamp ---
    if mode == "index":
        # 0..N-1 imaginary index
        new_ts = pd.RangeIndex(start=0, stop=len(df), step=1)
        df.insert(0, "ts", new_ts)
    elif mode == "calendar":
        # calendar timeline (start + freq), default periods = number of rows
        periods = len(df)
        new_ts = pd.date_range(start=start, periods=periods, freq=freq)
        df.insert(0, "timestamp", new_ts)
    else:
        raise ValueError("mode must be 'calendar' or 'index'.")

    # --- Save or return ---
    if csv_out:
        csv_out = Path(csv_out)
        df.to_csv(csv_out, index=False)
        return csv_out
    return df


In [7]:
relabel_energyplus(
    "idf-Oct\HVACDetailed-Oct-13_\eplusmtr.csv",
    csv_out=None,
    mode="calendar",
    start="2023-01-01",
    freq="H",
    expected_periods=365*24
)

C:\Users\s2589602\AppData\Local\Temp\ipykernel_12048\1575773852.py:55: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  new_ts = pd.date_range(start=start, periods=periods, freq=freq)


,timestamp,CoolingCoils:EnergyTransfer [J](Hourly),HeatingCoils:EnergyTransfer [J](Hourly),Electricity:HVAC [J](Hourly),Electricity:Plant [J](Hourly),Baseboard:EnergyTransfer [J](Hourly),Electricity:Facility [J](Hourly),DistrictHeatingWater:Facility [J](Hourly),DistrictCooling:Facility [J](Hourly),Cooling:Electricity [J](Hourly)
0,2023-01-01 00:00:00,0.0,1.599897e+08,1.049507e+07,4.238316e+06,2.095775e+08,3.470677e+07,3.654804e+08,0.0,0.0
1,2023-01-01 01:00:00,0.0,1.619460e+08,1.058960e+07,4.272544e+06,2.118490e+08,3.483553e+07,3.695091e+08,0.0,0.0
2,2023-01-01 02:00:00,0.0,1.648987e+08,1.072615e+07,4.316645e+06,2.136131e+08,3.501618e+07,3.742018e+08,0.0,0.0
3,2023-01-01 03:00:00,0.0,1.674395e+08,1.083382e+07,4.355298e+06,2.151904e+08,3.516251e+07,3.783296e+08,0.0,0.0
4,2023-01-01 04:00:00,0.0,1.692886e+08,1.091170e+07,4.382213e+06,2.152377e+08,3.526730e+07,3.803149e+08,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
8755,2023-12-31 19:00:00,0.0,8.695541e+07,6.222369e+06,2.754575e+06,1.753990e+08,2.895033e+07,2.608106e+08,0.0,0.0
8756,2023-12-31 20:00:00,0.0,9.382294e+07,6.774152e+06,2.945016e+06,1.776916e+08,2.969255e+07,2.694495e+08,0.0,0.0
8757,2023-12-31 21:00:00,0.0,1.025641e+08,7.354125e+06,3.151395e+06,1.819275e+08,3.047891e+07,2.820608e+08,0.0,0.0
8758,2023-12-31 22:00:00,0.0,1.103563e+08,7.886068e+06,3.330027e+06,1.845242e+08,3.118948e+07,2.921558e+08,0.0,0.0


In [ ]:
import pandas as pd

def compute_eui_table(df: pd.DataFrame, area_m2: float) -> pd.DataFrame:
    """
    Compute annual energy (kWh) and EUI (kWh/m²·yr) for selected EnergyPlus output columns.
    Expects hourly energy columns in Joules.
    """
    # 1 kWh = 3.6e6 J
    J_TO_KWH = 1 / 3_600_000

    # The exact column names you mentioned
    target_cols = [
        "Electricity:HVAC [J](Hourly)",
        "Electricity:Plant [J](Hourly)",
        "Baseboard:EnergyTransfer [J](Hourly)",
        "Electricity:Facility [J](Hourly)",
        "DistrictHeatingWater:Facility [J](Hourly)",
        "DistrictCooling:Facility [J](Hourly)",
        "Cooling:Electricity [J](Hourly)",
    ]

    present = [c for c in target_cols if c in df.columns]
    missing = [c for c in target_cols if c not in df.columns]
    if missing:
        print("⚠️ Missing columns (skipped):", "; ".join(missing))

    out = []
    for col in present:
        annual_J = df[col].sum(skipna=True)
        annual_kWh = annual_J * J_TO_KWH
        eui = annual_kWh / area_m2 if area_m2 > 0 else float("nan")
        out.append({
            "Metric": col,
            "Annual Energy (kWh)": annual_kWh,
            "EUI (kWh/m²·yr)": eui
        })

    # Optional: include a convenience “Total Facility Electricity EUI” if present
    # (Do NOT sum 'Facility' with sub-end uses—double counting!)
    # If you want a total site EUI, prefer the facility-level totals that represent the whole building.
    return pd.DataFrame(out).sort_values("Metric").reset_index(drop=True)

# --- Example usage ---
df = relabel_energyplus("idf-Oct/HVACDetailed-Oct-13_/eplusmtr.csv",
                        csv_out=None, mode="calendar",
                        start="2023-01-01", freq="H",
                        expected_periods=365*24)
#
area_m2 = 9600
eui_table = compute_eui_table(df, area_m2)
print(eui_table.to_string(index=False))

                                   Metric  Annual Energy (kWh)  EUI (kWh/m²·yr)
     Baseboard:EnergyTransfer [J](Hourly)        195622.329913        20.377326
          Cooling:Electricity [J](Hourly)         26266.058290         2.736048
     DistrictCooling:Facility [J](Hourly)             0.000000         0.000000
DistrictHeatingWater:Facility [J](Hourly)        228962.246248        23.850234
         Electricity:Facility [J](Hourly)        610987.581334        63.644540
             Electricity:HVAC [J](Hourly)         49387.210379         5.144501
            Electricity:Plant [J](Hourly)         60437.158645         6.295537


C:\Users\s2589602\AppData\Local\Temp\ipykernel_12048\1575773852.py:55: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  new_ts = pd.date_range(start=start, periods=periods, freq=freq)
